In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('../data/superstore_clean.csv')
df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,sub_category,product_name,sales,quantity,discount,profit,order_week,year,month,week_of_year
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2016-11-07,2016,11,45
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2016-11-07,2016,11,45
2,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2015-10-05,2015,10,41
3,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694,2014-06-09,2014,6,24
4,11,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092,2014-06-09,2014,6,24


In [3]:
# Force conversion
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# Optional: drop rows where conversion failed
df = df.dropna(subset=['order_date'])

Grain: Week × Region <br>Metrics: Total Sales, Total Profit, Total Quantity

In [4]:
exec_overview = (
    df.groupby([pd.Grouper(key='order_date', freq='W'), 'region'])
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          total_quantity=('quantity', 'sum')
      )
      .reset_index()
)

exec_overview.head()

,order_date,region,total_sales,total_profit,total_quantity
0,2014-01-12,Central,76.728,-53.7096,3
1,2014-01-12,East,9.940,3.0814,2
2,2014-01-12,South,2625.760,767.7032,10
3,2014-01-19,East,189.064,-24.6872,10
4,2014-01-19,South,545.940,87.3504,6


Grain: Week × Region × Category <br> Metrics: Total Sales (target), Quantity, Discount, Profit

In [5]:
forecast_table = (
    df.groupby([pd.Grouper(key='order_date', freq='W'), 'region', 'category'])
      .agg(
          weekly_sales=('sales', 'sum'),
          total_quantity=('quantity', 'sum'),
          avg_discount=('discount', 'mean'),
          total_profit=('profit', 'sum')
      )
      .reset_index()
)

forecast_table.head()

,order_date,region,category,weekly_sales,total_quantity,avg_discount,total_profit
0,2014-01-12,Central,Furniture,76.728,3,0.60,-53.7096
1,2014-01-12,East,Furniture,9.940,2,0.00,3.0814
2,2014-01-12,South,Furniture,2625.760,10,0.00,767.7032
3,2014-01-19,East,Furniture,189.064,10,0.35,-24.6872
4,2014-01-19,South,Furniture,545.940,6,0.00,87.3504


Grain: Region × Category × Sub-Category <br> Metrics: Total Sales, Total Profit, Avg Discount, Profit Margin

In [6]:
profit_table = (
    df.groupby(['region', 'category', 'sub_category'])
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_discount=('discount', 'mean')
      )
      .reset_index()
)

profit_table['profit_margin'] = profit_table['total_profit'] / profit_table['total_sales']

profit_table.head()

,region,category,sub_category,total_sales,total_profit,avg_discount,profit_margin
0,Central,Furniture,Bookcases,24157.1768,-1997.9043,0.232800,-0.082704
1,Central,Furniture,Chairs,85230.6460,6592.7221,0.192857,0.077352
2,Central,Furniture,Furnishings,15254.3700,-3906.2168,0.403902,-0.256072
3,Central,Furniture,Tables,39154.9710,-3559.6504,0.262500,-0.090912
4,East,Furniture,Bookcases,43819.3340,-1167.6318,0.220000,-0.026646


In [7]:
exec_overview.to_csv("../data/exec_overview.csv", index=False)
forecast_table.to_csv("../data/forecast_table.csv", index=False)
profit_table.to_csv("../data/profit_table.csv", index=False)


In [8]:
forecast_df = pd.read_csv(
    "../data/forecast_table.csv",
    parse_dates=['order_date']
)

forecast_df.sort_values(
    by=['region', 'category', 'order_date'],
    inplace=True
)

In [9]:
forecast_df['year'] = forecast_df['order_date'].dt.year
forecast_df['month'] = forecast_df['order_date'].dt.month
forecast_df['week_of_year'] = forecast_df['order_date'].dt.isocalendar().week.astype(int)

forecast_df.head()


,order_date,region,category,weekly_sales,total_quantity,avg_discount,total_profit,year,month,week_of_year
0,2014-01-12,Central,Furniture,76.728,3,0.600,-53.7096,2014,1,2
6,2014-01-26,Central,Furniture,429.630,8,0.000,74.6331,2014,1,4
11,2014-02-16,Central,Furniture,393.830,4,0.000,68.4929,2014,2,7
14,2014-02-23,Central,Furniture,45.480,9,0.300,-4.4132,2014,2,8
15,2014-03-02,Central,Furniture,1436.427,21,0.375,-250.1004,2014,3,9


Creating Lag Features

In [10]:
for lag in [1, 4, 12]:
    forecast_df[f'sales_lag_{lag}'] = (
        forecast_df
        .groupby(['region', 'category'])['weekly_sales']
        .shift(lag)
    )

Creating Rolling Statistics

In [11]:
forecast_df['rolling_mean_4'] = (
    forecast_df
    .groupby(['region', 'category'])['weekly_sales']
    .shift(1)
    .rolling(window=4)
    .mean()
)

forecast_df['rolling_mean_8'] = (
    forecast_df
    .groupby(['region', 'category'])['weekly_sales']
    .shift(1)
    .rolling(window=8)
    .mean()
)


In [12]:
forecast_df = forecast_df.dropna().reset_index(drop=True)
forecast_df.to_csv(
    "../data/forecast_features.csv",
    index=False
)


In [13]:
target = 'weekly_sales'

feature_cols = [
    'year', 'month', 'week_of_year',
    'sales_lag_1', 'sales_lag_4', 'sales_lag_12',
    'rolling_mean_4', 'rolling_mean_8'
]


In [14]:
train = forecast_df[forecast_df['order_date'] < '2018-01-01']
test  = forecast_df[forecast_df['order_date'] >= '2018-01-01']


In [15]:
test['naive_forecast'] = test['sales_lag_1']


In [17]:
import sys
!{sys.executable} -m pip install xgboost==1.7.6


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s  0:00:00


In [19]:
from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    random_state=42
)

_ = model.fit(train[feature_cols], train[target])
